# Merge report and patent

## Find out those green patent without report

In [19]:
import pandas as pd

In [26]:
report = pd.read_csv("../CSR Reporting/Dataset/report_company_without_overlap.csv")
y = pd.read_csv("../Green_patent_dataset/merged_dataset/forward_count_value_ticker_all_y.csv")

In [31]:
y = y[['ticker', 'year']]
y

,ticker,year
0,A,2004
1,A,2006
2,A,2007
3,A,2008
4,A,2012
...,...,...
2658,XXIA,2014
2659,ZTS,2009
2660,ZTS,2013
2661,ZTS,2014


In [30]:
report['lagged_year'] = report['year'] + 1
report[['ticker', 'year']]

,ticker,year
0,AAL,2007
1,AAL,2008
2,AAL,2009
3,AAL,2011
4,AAL,2012
...,...,...
6407,hpq,2017
6408,hpq,2018
6409,hpq,2019
6410,hpq,2020


In [32]:
# 找出 y 中有，但 report 中沒有的組合
y_not_in_report = y.merge(report, on=['ticker', 'year'], how='left', indicator=True)
y_not_in_report = y_not_in_report[y_not_in_report['_merge'] == 'left_only']
y_not_in_report = y_not_in_report.drop(columns=['_merge'])

# 顯示結果
print(f"共有 {len(y_not_in_report)} 筆資料在 y 中但不在 report 中：")
print(y_not_in_report)

共有 2290 筆資料在 y 中但不在 report 中：
     ticker  year folder_name exchange  lagged_year company_name
6      AAPL  2004         NaN      NaN          NaN          NaN
7      AAPL  2005         NaN      NaN          NaN          NaN
8      AAPL  2006         NaN      NaN          NaN          NaN
9      AAPL  2007         NaN      NaN          NaN          NaN
10     AAPL  2008         NaN      NaN          NaN          NaN
...     ...   ...         ...      ...          ...          ...
2673   XXIA  2014         NaN      NaN          NaN          NaN
2674    ZTS  2009         NaN      NaN          NaN          NaN
2675    ZTS  2013         NaN      NaN          NaN          NaN
2676    ZTS  2014         NaN      NaN          NaN          NaN
2677    ZTS  2017         NaN      NaN          NaN          NaN

[2290 rows x 6 columns]


In [33]:
y_not_in_report.to_csv("y_not_in_report.csv", index=False)

In [3]:
import pandas as pd
y_not_in_report = pd.read_csv("y_not_in_report.csv")
company = pd.read_csv("../CSR Reporting/Dataset/company_name_all_ticker.csv")
y_not_in_report_unique = y_not_in_report['ticker'].unique()
y_not_in_report_unique = pd.DataFrame(y_not_in_report_unique, columns=['ticker'])
y_not_in_report_unique = y_not_in_report_unique.merge(company, on='ticker', how='left')
# y_not_in_report_unique = y_not_in_report_unique.drop(columns=['ticker'])
# 刪除空的列
y_not_in_report_unique = y_not_in_report_unique.dropna()
y_not_in_report_unique.to_csv("y_not_in_report_unique.csv", index=False)

## Merge report and forward, count

In [1]:
import pandas as pd
import numpy as np

In [2]:
report = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_company_without_overlap.csv")
forward_count_ticker = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker.csv")
value = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/green_patents_values_filing_ticker.csv")

In [3]:
report['ticker'] = report['ticker'].str.upper()
value['TICKER'] = value['TICKER'].str.upper()
forward_count_ticker['TICKER'] = forward_count_ticker['TICKER'].str.upper()

In [4]:
forward_count_ticker = forward_count_ticker.drop_duplicates()
forward_count_ticker

,gvkey_numeric,filing_year,total_5yr_forward_citations,within_5_years,patents_count,GVKEY,PERMNO,CUSIP,TICKER
0,4839,2004,0,0.0,3,4839.0,25785.0,34537086,F
1,4839,2005,0,0.0,6,4839.0,25785.0,34537086,F
2,4839,2006,4,2.0,5,4839.0,25785.0,34537086,F
3,4839,2007,280,24.0,28,4839.0,25785.0,34537086,F
4,4839,2008,358,27.0,34,4839.0,25785.0,34537086,F
...,...,...,...,...,...,...,...,...,...
8760,23051,2014,3,1.0,1,23051.0,76393.0,60474510,MIRR
8761,65944,2011,0,0.0,1,65944.0,85686.0,04546C20,DPMD
8762,65944,2011,0,0.0,1,65944.0,85686.0,04546C20,DMI
8763,65944,2011,0,0.0,1,65944.0,85686.0,04546C20,DEPO


In [5]:
value = value.drop_duplicates()
value['year'] = value['year'].astype(int)
value

,permno,year,total_values_nominal,patents_count_nominal,total_values_real,patents_count_real,value_per_patent_nominal,value_per_patent_real,GVKEY,PERMNO,CUSIP,TICKER,COMNAM,EXCHANGE
0,10032,1997,4.634948,1,2.454536,1,4.634948,2.454536,12945.0,10032.0,72913210,PLXS,PLEXUS CORP,NAS
1,10078,1989,29.746362,2,20.459795,2,14.873181,10.229898,12136.0,10078.0,86681020,JAVA,SUN MICROSYSTEMS INC,NAS
2,10078,1991,32.915205,1,22.639352,1,32.915205,22.639352,12136.0,10078.0,86681020,JAVA,SUN MICROSYSTEMS INC,NAS
3,10078,1995,242.379096,3,148.096757,3,80.793032,49.365586,12136.0,10078.0,86681020,JAVA,SUN MICROSYSTEMS INC,NAS
4,10078,1996,463.690059,8,268.795036,8,57.961257,33.599379,12136.0,10078.0,86681020,JAVA,SUN MICROSYSTEMS INC,NAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10697,93436,2015,1115.396654,4,452.246280,4,278.849163,113.061570,184996.0,93436.0,88160R10,TSLA,TESLA INC,NAS
10698,93436,2016,573.500674,1,217.818401,1,573.500674,217.818401,184996.0,93436.0,88160R10,TSLA,TESLA INC,NAS
10699,93436,2019,6813.554617,2,2167.926984,2,3406.777308,1083.963492,184996.0,93436.0,88160R10,TSLA,TESLA INC,NAS
10700,93436,2021,4826.775018,2,1535.776316,2,2413.387509,767.888158,184996.0,93436.0,88160R10,TSLA,TESLA INC,NAS


In [6]:
forward_count_value_ticker = pd.merge(forward_count_ticker, value[['TICKER', 'year', 'total_values_nominal', 'total_values_real', 'value_per_patent_nominal','value_per_patent_real']], how='left', left_on=['TICKER', 'filing_year'], right_on=['TICKER', 'year'])
forward_count_value_ticker.dropna(inplace=True)
forward_count_value_ticker

,gvkey_numeric,filing_year,total_5yr_forward_citations,within_5_years,patents_count,GVKEY,PERMNO,CUSIP,TICKER,year,total_values_nominal,total_values_real,value_per_patent_nominal,value_per_patent_real
0,4839,2004,0,0.0,3,4839.0,25785.0,34537086,F,2004.0,2412.696994,1134.299961,18.703077,8.793023
1,4839,2005,0,0.0,6,4839.0,25785.0,34537086,F,2005.0,3609.208193,1641.743348,24.386542,11.092860
2,4839,2006,4,2.0,5,4839.0,25785.0,34537086,F,2006.0,5717.632772,2572.015049,30.252025,13.608545
3,4839,2007,280,24.0,28,4839.0,25785.0,34537086,F,2007.0,6725.882615,2979.284270,35.214045,15.598347
4,4839,2008,358,27.0,34,4839.0,25785.0,34537086,F,2008.0,4464.263668,1947.394325,29.761758,12.982629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8720,26162,2014,14,1.0,1,26162.0,16562.0,03272410,ANAB,2014.0,51.922721,20.556447,51.922721,20.556447
8729,9590,2016,6,1.0,1,9590.0,41574.0,46121H10,IIN,2016.0,1.773701,0.702216,1.773701,0.702216
8732,9815,2014,2,1.0,1,9815.0,71176.0,83549510,SON,2014.0,7.149726,2.830610,7.149726,2.830610
8751,187215,2016,1,1.0,1,187215.0,12792.0,37951D30,ENT,2016.0,1.495850,0.581114,1.495850,0.581114


In [7]:
# 標準化 ticker 為大寫（確保一致）
report['ticker'] = report['ticker'].str.upper()
forward_count_value_ticker['TICKER'] = forward_count_value_ticker['TICKER'].str.upper()

# 取交集
common_tickers = set(report['ticker']).intersection(set(forward_count_value_ticker['TICKER']))

# 轉成 DataFrame 方便查看
common_ticker_df = pd.DataFrame(sorted(common_tickers), columns=['TICKER'])

# 顯示結果
print(f"✅ 共有 {len(common_ticker_df)} 個 ticker 同時出現在 report 和 forward_count_value_ticker 中")
print(common_ticker_df.head())


✅ 共有 230 個 ticker 同時出現在 report 和 forward_count_value_ticker 中
  TICKER
0      A
1   AAPL
2   ABBV
3    ABT
4   ACCO


In [8]:
# Step 1: 先合併（與原始邏輯一致）
report_forward_count_value = pd.merge(
    report,
    forward_count_value_ticker[['TICKER', 'filing_year', 'total_5yr_forward_citations', 'within_5_years', 'patents_count', 'total_values_nominal', 'total_values_real', 'value_per_patent_nominal','value_per_patent_real']],
    how='left',
    left_on=['ticker', 'year'],  # 根據你的欄位名稱應該是 year1
    right_on=['TICKER', 'filing_year']
)

# Step 2: 建立一個 ticker 的交集集合
common_tickers = set(report['ticker']).intersection(set(forward_count_value_ticker['TICKER']))

# Step 3: 建立條件：在交集內的 ticker，但數值為 NaN（表示年份沒對上）
mask = (
    report_forward_count_value['ticker'].isin(common_tickers) &
    report_forward_count_value['total_5yr_forward_citations'].isna()
)

# Step 4: 對這些條件補上 0
cols_to_fill = ['total_5yr_forward_citations', 'within_5_years', 'patents_count', 'total_values_nominal', 'total_values_real', 'value_per_patent_nominal','value_per_patent_real']
report_forward_count_value.loc[mask, cols_to_fill] = 0

# Step 5: 此時只補了你要補的，其他 NaN 不動，接著轉型時只針對非 NaN 行
for col in cols_to_fill:
    report_forward_count_value[col] = report_forward_count_value[col].apply(lambda x: int(x) if pd.notna(x) else np.nan)

print("✅ 已補上 forward 缺年份的 0，其他保持 NaN")

✅ 已補上 forward 缺年份的 0，其他保持 NaN


In [9]:
report_forward_count_value

,folder_name,exchange_x,ticker,year,company_name,exchange_y,TICKER,filing_year,total_5yr_forward_citations,within_5_years,patents_count,total_values_nominal,total_values_real,value_per_patent_nominal,value_per_patent_real
0,NASDAQ_AAL_2007,NASDAQ,AAL,2007,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NASDAQ_AAL_2008,NASDAQ,AAL,2008,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NASDAQ_AAL_2009,NASDAQ,AAL,2009,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NASDAQ_AAL_2011,NASDAQ,AAL,2011,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NASDAQ_AAL_2012,NASDAQ,AAL,2012,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6070,NYSE_hpq_2017,NYSE,HPQ,2017,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6071,NYSE_hpq_2018,NYSE,HPQ,2018,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6072,NYSE_hpq_2019,NYSE,HPQ,2019,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6073,NYSE_hpq_2020,NYSE,HPQ,2020,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Step 1: 合併 report 和 forward 資料
report_forward_count_value_status = pd.merge(
    report,
    report_forward_count_value[['TICKER', 'filing_year', 'total_5yr_forward_citations', 'within_5_years', 'patents_count', 'total_values_nominal', 'total_values_real', 'value_per_patent_nominal','value_per_patent_real']],
    how='left',
    left_on=['ticker', 'year'],
    right_on=['TICKER', 'filing_year']
)

# Step 2: 定義 ticker 的交集
common_tickers = set(report['ticker']).intersection(set(forward_count_ticker['TICKER']))

# Step 3: 建立 match_status 欄位
def classify_match(row):
    if pd.notna(row['total_5yr_forward_citations']):
        return 'full_match'
    elif row['ticker'] in common_tickers:
        return 'missing_year'
    else:
        return 'ticker_not_found'

report_forward_count_value_status['match_status'] = report_forward_count_value_status.apply(classify_match, axis=1)

# Step 4: 僅對 missing_year 的資料補 0
cols_to_fill = ['total_5yr_forward_citations', 'within_5_years', 'patents_count', 'total_values_nominal', 'total_values_real', 'value_per_patent_nominal','value_per_patent_real']
report_forward_count_value_status.loc[report_forward_count_value_status['match_status'] == 'missing_year', cols_to_fill] = 0

# Step 5: 將補過的值轉 int，其餘仍為 NaN
for col in cols_to_fill:
    report_forward_count_value_status[col] = report_forward_count_value_status[col].apply(lambda x: int(x) if pd.notna(x) else np.nan)

# Done!
print("✅ match_status 欄位建立完成，forward 資訊依狀況正確補值")
print(report_forward_count_value_status[['ticker', 'year', 'match_status']].value_counts())


✅ match_status 欄位建立完成，forward 資訊依狀況正確補值
ticker  year  match_status    
GM      2016  full_match          8
        2017  full_match          8
APTV    2022  missing_year        5
        2018  missing_year        5
        2020  missing_year        5
                                 ..
EXLS    2020  ticker_not_found    1
        2019  ticker_not_found    1
EXC     2021  missing_year        1
        2020  missing_year        1
EXTN    2020  ticker_not_found    1
Name: count, Length: 5909, dtype: int64


In [11]:
# print ticker_not_found
ticker_not_found = report_forward_count_value_status[report_forward_count_value_status['match_status'] == 'ticker_not_found']
ticker_not_found

,folder_name,exchange_x,ticker,year,company_name,exchange_y,TICKER,filing_year,total_5yr_forward_citations,within_5_years,patents_count,total_values_nominal,total_values_real,value_per_patent_nominal,value_per_patent_real,match_status
0,NASDAQ_AAL_2007,NASDAQ,AAL,2007,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found
1,NASDAQ_AAL_2008,NASDAQ,AAL,2008,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found
2,NASDAQ_AAL_2009,NASDAQ,AAL,2009,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found
3,NASDAQ_AAL_2011,NASDAQ,AAL,2011,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found
4,NASDAQ_AAL_2012,NASDAQ,AAL,2012,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6086,NYSE_hpq_2017,NYSE,HPQ,2017,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found
6087,NYSE_hpq_2018,NYSE,HPQ,2018,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found
6088,NYSE_hpq_2019,NYSE,HPQ,2019,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found
6089,NYSE_hpq_2020,NYSE,HPQ,2020,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ticker_not_found


In [12]:
report_forward_count_value_dropna = report_forward_count_value.dropna(subset=['total_5yr_forward_citations'])
report_forward_count_value_dropna.drop(columns=['filing_year', 'exchange_y', 'TICKER'], inplace=True)
report_forward_count_value_dropna.drop_duplicates(inplace=True)

/tmp/ipykernel_58901/467127963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_forward_count_value_dropna.drop(columns=['filing_year', 'exchange_y', 'TICKER'], inplace=True)
/tmp/ipykernel_58901/467127963.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_forward_count_value_dropna.drop_duplicates(inplace=True)


In [13]:
report_forward_count_value_dropna.to_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_forward_count_value.csv", index=False)

## Count how many count company with report

In [67]:
import pandas as pd

In [68]:
forward_count_within = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker_within_report.csv")

In [69]:
len(forward_count_within['TICKER'].unique())

602

# report_forward_count_value EDA

In [54]:
import pandas as pd

In [55]:
report_forward_count_value = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_forward_count_value.csv")

In [56]:
len(report_forward_count_value['ticker'].unique())

230

In [57]:
report_forward_count_value['year'].min()

np.int64(1998)

In [58]:
report_forward_count_value['year'].max()

np.int64(2022)

In [59]:
report_forward_count_value

,folder_name,exchange_x,ticker,year,company_name,total_5yr_forward_citations,within_5_years,patents_count,total_values_nominal,total_values_real,value_per_patent_nominal,value_per_patent_real
0,NASDAQ_AAPL_2014,NASDAQ,AAPL,2014,Apple Inc.,910.0,121.0,122.0,11322.0,4498.0,76.0,30.0
1,NASDAQ_AAPL_2015,NASDAQ,AAPL,2015,Apple Inc.,892.0,120.0,123.0,12050.0,4672.0,86.0,33.0
2,NASDAQ_AAPL_2016,NASDAQ,AAPL,2016,Apple Inc.,801.0,93.0,96.0,11970.0,4542.0,105.0,40.0
3,NASDAQ_AAPL_2017,NASDAQ,AAPL,2017,Apple Inc.,381.0,86.0,88.0,9283.0,3391.0,140.0,51.0
4,NASDAQ_AAPL_2018,NASDAQ,AAPL,2018,Apple Inc.,144.0,26.0,29.0,12956.0,4707.0,166.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1568,NYSE_XRX_2016,NYSE,XRX,2016,Xerox Corp,56.0,6.0,6.0,12.0,4.0,4.0,1.0
1569,NYSE_XRX_2017,NYSE,XRX,2017,Xerox Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1570,NYSE_XRX_2018,NYSE,XRX,2018,Xerox Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1571,NYSE_XRX_2019,NYSE,XRX,2019,Xerox Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
# 計算每家企業的資料筆數
company_counts = report_forward_count_value['company_name'].value_counts()

# 計算平均
average_reports_per_company = company_counts.mean()

print(f"✅ 每家企業平均有 {average_reports_per_company:.2f} 筆資料")
print(company_counts.describe())

✅ 每家企業平均有 6.65 筆資料
count    232.000000
mean       6.650862
std        4.718353
min        1.000000
25%        3.000000
50%        5.000000
75%        9.000000
max       23.000000
Name: count, dtype: float64


In [61]:
# Step 1: 找出三個欄位全為 0 的公司名稱
all_zero_companies = report_forward_count_value.groupby('company_name')[['total_5yr_forward_citations', 'within_5_years', 'patents_count', 'total_values_nominal', 'total_values_real', 'value_per_patent_nominal','value_per_patent_real']] \
    .apply(lambda g: ((g == 0) | (g.isna())).all().all())  # 檢查每一列、每個欄是否全為 0 或 NaN

# 篩出需移除的公司名稱
companies_to_remove = all_zero_companies[all_zero_companies].index

# Step 2: 將這些公司從原資料刪除
filtered_report_forward_count_value = report_forward_count_value[~report_forward_count_value['company_name'].isin(companies_to_remove)]

# Step 3: 印出刪除前後差異
print(f"📊 原本共有 {len(report_forward_count_value)} 筆資料")
print(f"❌ 移除掉 {len(report_forward_count_value) - len(filtered_report_forward_count_value)} 筆（來自 {len(companies_to_remove)} 家所有數值皆為 0 的公司）")
print(f"✅ 現在剩下 {len(filtered_report_forward_count_value)} 筆資料")


📊 原本共有 1573 筆資料
❌ 移除掉 469 筆（來自 118 家所有數值皆為 0 的公司）
✅ 現在剩下 1104 筆資料


In [44]:
len(report_forward_count_value['ticker'].unique())

392

In [45]:
company_counts_filtered = report_forward_count_value['company_name'].value_counts()
company_counts_filtered.mean()

np.float64(6.315656565656566)

In [46]:
report_forward_count_value

,folder_name,exchange_x,ticker,year,company_name,total_5yr_forward_citations,within_5_years,patents_count
0,NASDAQ_AAPL_2014,NASDAQ,AAPL,2014,Apple Inc.,910.0,121.0,122.0
1,NASDAQ_AAPL_2015,NASDAQ,AAPL,2015,Apple Inc.,892.0,120.0,123.0
2,NASDAQ_AAPL_2016,NASDAQ,AAPL,2016,Apple Inc.,801.0,93.0,96.0
3,NASDAQ_AAPL_2017,NASDAQ,AAPL,2017,Apple Inc.,381.0,86.0,88.0
4,NASDAQ_AAPL_2018,NASDAQ,AAPL,2018,Apple Inc.,144.0,26.0,29.0
...,...,...,...,...,...,...,...,...
2546,NYSE_ZMH_2017,NYSE,ZMH,2017,Zimmer Holdings Inc.,0.0,0.0,0.0
2547,NYSE_ZMH_2018,NYSE,ZMH,2018,Zimmer Holdings Inc.,0.0,0.0,0.0
2548,NYSE_ZMH_2019,NYSE,ZMH,2019,Zimmer Holdings Inc.,0.0,0.0,0.0
2549,NYSE_ZMH_2020,NYSE,ZMH,2020,Zimmer Holdings Inc.,0.0,0.0,0.0


# Merge embedding and patent

In [18]:
import pandas as pd

In [19]:
report_forward_count_value = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_forward_count_value.csv")
csr_embeddings = pd.read_csv("/opt/hdd_1/research_hub/csr_project/output_dataset/csr_embeddings.csv")

In [20]:
print(report_forward_count_value.columns)
print(csr_embeddings.columns)

Index(['folder_name', 'exchange_x', 'ticker', 'year', 'company_name',
       'total_5yr_forward_citations', 'within_5_years', 'patents_count',
       'total_values_nominal', 'total_values_real', 'value_per_patent_nominal',
       'value_per_patent_real'],
      dtype='object')
Index(['file_name', 'dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5',
       'dim_6', 'dim_7', 'dim_8',
       ...
       'dim_1014', 'dim_1015', 'dim_1016', 'dim_1017', 'dim_1018', 'dim_1019',
       'dim_1020', 'dim_1021', 'dim_1022', 'dim_1023'],
      dtype='object', length=1025)


In [21]:
csr_embeddings_forward_count = pd.merge(
    report_forward_count_value[['folder_name', 'ticker', 'year','total_5yr_forward_citations','patents_count', 'total_values_real']],
    csr_embeddings,
    how='left',
    left_on=['folder_name'],
    right_on=['file_name']
)
csr_embeddings_forward_count.drop(columns=['file_name'], inplace=True)

In [24]:
csr_embeddings_forward_count.head(10)

,folder_name,ticker,year,total_5yr_forward_citations,patents_count,total_values_real,dim_0,dim_1,dim_2,dim_3,...,dim_1014,dim_1015,dim_1016,dim_1017,dim_1018,dim_1019,dim_1020,dim_1021,dim_1022,dim_1023
0,NASDAQ_AAPL_2014,AAPL,2014,910.0,122.0,4498.0,-0.091830,0.293040,-0.179719,0.083607,...,-0.086017,-0.403820,0.282563,0.331965,0.290067,-0.223406,-0.325722,0.009091,-0.170071,0.163041
1,NASDAQ_AAPL_2015,AAPL,2015,892.0,123.0,4672.0,-0.085034,0.276596,-0.139275,0.068203,...,-0.090395,-0.406836,0.278167,0.320248,0.292318,-0.212523,-0.331629,-0.030166,-0.150414,0.180043
2,NASDAQ_AAPL_2016,AAPL,2016,801.0,96.0,4542.0,-0.067159,0.277524,-0.148442,0.092723,...,-0.109939,-0.416270,0.282748,0.334449,0.296767,-0.216122,-0.327691,-0.037364,-0.151454,0.178229
3,NASDAQ_AAPL_2017,AAPL,2017,381.0,88.0,3391.0,-0.053141,0.259755,-0.160064,0.098964,...,-0.110439,-0.411443,0.307153,0.298774,0.284501,-0.213012,-0.317366,-0.034005,-0.147262,0.175490
4,NASDAQ_AAPL_2018,AAPL,2018,144.0,29.0,4707.0,-0.070878,0.338528,-0.095608,0.093072,...,-0.099431,-0.410019,0.305583,0.368817,0.274357,-0.190008,-0.347579,-0.004540,-0.125971,0.124562
5,NASDAQ_AAPL_2019,AAPL,2019,0.0,2.0,6197.0,-0.027629,0.314667,-0.098202,0.089688,...,-0.127325,-0.396108,0.332946,0.378781,0.298330,-0.213221,-0.337118,-0.008280,-0.156933,0.140300
6,NASDAQ_AAPL_2021,AAPL,2021,0.0,0.0,0.0,0.057423,0.316942,-0.129241,0.005188,...,-0.088056,-0.318386,0.303288,0.311286,0.449222,-0.203371,-0.198375,-0.083905,-0.113969,0.202417
7,NASDAQ_ADBE_2013,ADBE,2013,0.0,0.0,0.0,-0.112048,0.372203,-0.122453,0.006096,...,-0.000974,-0.416418,0.202442,0.281165,0.256865,-0.252488,-0.316124,0.062103,-0.074278,0.165792
8,NASDAQ_ADBE_2014,ADBE,2014,4.0,1.0,12.0,-0.098989,0.344149,-0.103529,0.113674,...,-0.105083,-0.416937,0.263158,0.250662,0.253440,-0.296858,-0.371066,0.088189,-0.062360,0.187736
9,NASDAQ_ADBE_2015,ADBE,2015,8.0,1.0,32.0,-0.098959,0.309574,-0.095862,-0.045760,...,-0.026325,-0.422132,0.282305,0.270916,0.298384,-0.286227,-0.393734,-0.005148,-0.047201,0.191042


In [23]:
csr_embeddings_forward_count.to_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/csr_embeddings_forward_count_value.csv", index=False)